<a href="https://colab.research.google.com/github/pgauravv/NLP_Predict_best_answer/blob/master/Predict_best_answer_GAV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# my thanks go to Baptiste + DP
# A-P
import os
import numpy as np
from scipy.sparse import csc_matrix, save_npz, load_npz
import re
import pickle
import pandas as pd
import spacy

from spacy.lang.en import English
from datetime import datetime
from gensim.corpora import Dictionary
from nltk.tokenize import TweetTokenizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from nltk.stem import PorterStemmer



#import matplotlib.pyplot as plt

In [4]:
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My Drive/NLP_Final_Project'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

# 18 fields expected.
field_names = ['id', 'type', 'is_best_answer', 'topic_id', 'parent_id', 'votes',
               'title', 'content', 'member', 'category', 'state', 'is_solved', 'num_answers',
               'country', 'date', 'last_answer_date', 'author_crc', 'visits']


In [0]:


txt_path = "/content/drive/My Drive/NLP_Final_Project/QA/export-forums_en.csv"
entity_path = "/content/drive/My Drive/NLP_Final_Project/QA/export-forums_en.pickle"
csv_path = "/content/drive/My Drive/NLP_Final_Project/QA/export-forums_en.format.csv"
data_path = "/content/drive/My Drive/NLP_Final_Project/QA/"



In [0]:

def format_entities():
    '''
    Read the raw data, format the list of entities, serialize them.
    '''

    def build_entities(txt_path, max_entities=None):
        '''
        Return a list of structured entities from raw txt file.
        '''
        # Read text file.
        with open(txt_path, 'r', encoding='utf8') as f:
            # Entities and current entity.
            entities, entity = [], {}
            # Entity values might be split over lines
            field_counter = 0
            # Process lines
            for line in f:
                # Prepare line
                line = line.replace("\\N", '"unkwown"')
                # Char start for extracted value.
                char_start = 1
                # Find values separators
                field_index = [m.start() for m in re.finditer('","', line)]
                # Browse value separators.
                for index in field_index:
                    # Extract in between value.
                    value = line[char_start:index]
                    # Update start index.
                    char_start = index + 3
                    # Update field counter.
                    field_counter += 1
                    # Update entity value.
                    try:
                        entity[field_names[field_counter-1]] += value
                    except KeyError:
                        entity[field_names[field_counter-1]] = value
                    except IndexError:
                        entity = {}
                        field_counter = 0
                # Content string is split.
                if field_counter == 7 and len(field_index) > 0:
                    entity[field_names[7]] = line[field_index[-1]:]
                    continue
                # Next content string.
                if field_counter == 7 and len(field_index) == 0:
                    entity[field_names[7]] += line
                    continue
                # Next entity.
                if len(entity) == 17:
                    field_counter = 0
                    entities.append(entity)
                    entity = {}
                    if max_entities is not None:
                        if len(entities) > max_entities:
                            return entities
        return entities

    # Write entities on disk.
    with open(entity_path, 'wb') as f:
        pickle.dump(build_entities(txt_path=txt_path, max_entities=None), f)
        print(f)


In [0]:


def entities_to_csv():
    '''
    Format entities to csv.
    '''
    with open(entity_path, 'rb') as obj:
        entities = pickle.load(obj)
    x = pd.DataFrame(entities)
    x.to_csv(csv_path)



In [0]:

def build_index():
    '''
    Build index and write.
    '''

    # Read entities.
    with open(entity_path, 'rb') as obj:
        entities = pickle.load(obj)

    # Build indexes
    user_index, question_index, answer_index, comment_index = {}, {}, {}, {}
    for e in entities:
        # Author
        if e['author_crc'] not in user_index:
            user_index[e['author_crc']] = len(user_index)
        # Questions
        if e['type'] == 'Q':
            if e['id'] not in question_index:
                question_index[e['id']] = len(question_index)
        # Answers
        if e['type'] == 'A':
            if e['id'] not in answer_index:
                answer_index[e['id']] = len(answer_index)
        # Answers
        if e['type'] == 'C':
            if e['id'] not in comment_index:
                comment_index[e['id']] = len(comment_index)

    # Write index.
    with open(os.path.join(data_path, 'user.index'), 'wb') as f:
        pickle.dump(user_index, f)
    with open(os.path.join(data_path, 'question.index'), 'wb') as f:
        pickle.dump(question_index, f)
    with open(os.path.join(data_path, 'answer.index'), 'wb') as f:
        pickle.dump(answer_index, f)
    with open(os.path.join(data_path, 'comment.index'), 'wb') as f:
        pickle.dump(comment_index, f)

    # Logs.
    print("Entities: ", len(entities))
    print("Users: ", len(user_index))
    print("Questions: ", len(question_index))
    print("Answers: ", len(answer_index))
    print("Comments: ", len(comment_index))



In [0]:

def read_indexes():
    '''
    Return user, question, answer and comment index.
    '''
    with open(os.path.join(data_path, 'user.index'), 'rb') as obj:
        user_index = pickle.load(obj)
    with open(os.path.join(data_path, 'question.index'), 'rb') as obj:
        question_index = pickle.load(obj)
    with open(os.path.join(data_path, 'answer.index'), 'rb') as obj:
        answer_index = pickle.load(obj)
    with open(os.path.join(data_path, 'comment.index'), 'rb') as obj:
        comment_index = pickle.load(obj)
    return user_index, question_index, answer_index, comment_index



In [0]:

def build_relations():
    '''
    Build UQ, UA, QA, UC, CA relations.
    '''

    # Read entities.
    with open(entity_path, 'rb') as obj:
        entities = pickle.load(obj)

    # Read indexes
    user_index, question_index, answer_index, comment_index = read_indexes()

    # Relations
    uq = []
    ua = []
    qa = []
    uc = []
    ca = []

    # Browse elements.
    for e in entities:

        # UQ
        if e['type'] == 'Q':
            u = user_index[e['author_crc']]
            q = question_index[e['id']]
            t = datetime.utcfromtimestamp(int(e['date'])).strftime('%Y-%m-%d %H:%M:%S')
            uq.append((u,q,t))

        # UA, QA
        if e['type'] == 'A':
            u = user_index[e['author_crc']]
            a = answer_index[e['id']]
            q = question_index[e['parent_id']]
            t = datetime.utcfromtimestamp(int(e['date'])).strftime('%Y-%m-%d %H:%M:%S')
            ua.append((u,a,t))
            qa.append((q,a,t))

        # UC, CA
        if e['type'] == 'C':
            try:
                u = user_index[e['author_crc']]
                c = comment_index[e['id']]
                a = answer_index[e['parent_id']]
                t = datetime.utcfromtimestamp(int(e['date'])).strftime('%Y-%m-%d %H:%M:%S')
                uc.append((u,c,t))
                ca.append((c,a,t))
            except KeyError:
                continue

    # Write relations.
    with open(os.path.join(data_path, 'uq.rel'), 'wb') as f:
        pickle.dump(uq, f)
    # Write relations.
    with open(os.path.join(data_path, 'ua.rel'), 'wb') as f:
        pickle.dump(ua, f)
    # Write relations.
    with open(os.path.join(data_path, 'qa.rel'), 'wb') as f:
        pickle.dump(qa, f)
    # Write relations.
    with open(os.path.join(data_path, 'uc.rel'), 'wb') as f:
        pickle.dump(uc, f)
    # Write relations.
    with open(os.path.join(data_path, 'ca.rel'), 'wb') as f:
        pickle.dump(ca, f)

    # Logs.
    print("uq: ", len(uq))
    print("ua: ", len(ua))
    print("qa: ", len(qa))
    print("uc: ", len(uc))
    print("ca: ", len(ca))



In [0]:

def build_vocabulary_and_corpus():
    '''
    Build the vocabularies and stem sequences for each type of entities.
    '''

    # Vocabulary (same for question and answers)
    v = Dictionary()

    # Stemmer.
    stemmer = PorterStemmer()

    # Tokenizer.
    tokenizer = TweetTokenizer()

    # Read indexes
    user_index, question_index, answer_index, comment_index = read_indexes()

    # Question, answer
    q = {}
    a = {}

    # Read entities.
    with open(entity_path, 'rb') as obj:
        entities = pickle.load(obj)

    # Browse question and answers to first build vocabulary.
    for e in entities:
        # Question or answer.
        if e['type'] == 'Q' or e['type'] == 'A':
            # String content.
            title = str(e['title']).encode('utf-8').lower()
            content = str(e['content']).encode('utf-8').lower()
            # Tokenize
            d = tokenizer.tokenize(title + content)
            # Stem word
            d = [stemmer.stem(s) for s in d]
            # Process vocabulary.
            v.add_documents([d])
            # Question
            if e['type'] == 'Q':
                q[question_index[e['id']]] = d
            # Answer
            if e['type'] == 'A':
                a[answer_index[e['id']]] = d

    # Write question corpus.
    with open(os.path.join(data_path, 'q.corpus'), 'wb') as f:
        pickle.dump(q, f)

    # Write answer corpus.
    with open(os.path.join(data_path, 'a.corpus'), 'wb') as f:
        pickle.dump(a, f)

    # Write to analyse.
    v.filter_extremes(no_below=1000, keep_n=10000)
    v.compactify()
    v.save(os.path.join(data_path, "raw_vocabulary.gensim"))


In [0]:

import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [0]:

def build_embeddings():
  # # Load the spacy model that you have installed
  # nlp = spacy.load('en_core_web_md')
  # # process a sentence using the model
  # doc = nlp("This is some text that I am processing with Spacy")
  # # It's that simple - all of the vectors and words are assigned after this point
  # # Get the vector for 'text':
  # doc[3].vector
  # # Get the mean vector for the entire sentence (useful for sentence classification etc.)
  # doc.vector
    print("Todo: your code goes here")


In [0]:

def build_models():
    print("Todo: your code goes here")    


In [0]:

# if __name__ == '__main__':
    
#     format_entities()
#     build_index()
#     build_relations()
#     build_vocabulary_and_corpus()

#     # build_embeddings()
#     # build_models()



In [0]:
 with open(os.path.join(data_path, 'uq.rel'), 'rb') as obj:
   uq = pickle.load(obj)

with open(os.path.join(data_path, 'ua.rel'), 'rb') as obj:
  ua = pickle.load(obj)

with open(os.path.join(data_path, 'qa.rel'), 'rb') as obj:
  qa = pickle.load(obj)

with open(os.path.join(data_path, 'uc.rel'), 'rb') as obj:
  ca = pickle.load(obj)
    
with open(os.path.join(data_path, 'ca.rel'), 'rb') as obj:
  ca = pickle.load(obj)

with open(os.path.join(data_path, 'q.corpus'), 'rb') as obj:
  q_corpus = pickle.load(obj)
    
with open(os.path.join(data_path, 'a.corpus'), 'rb') as obj:
  a_corpus = pickle.load(obj)

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [0]:
df_q_a=pd.read_csv(csv_path)

In [20]:
df_q_a.head()

,Unnamed: 0,id,type,is_best_answer,topic_id,parent_id,votes,title,content,member,category,state,is_solved,num_answers,country,date,last_answer_date,author_crc
0,0,69561,C,0,4,36102,0,Windows Vista to XP Downgrading/Reformat?,<a href='http://paparadit.blogspot.com/2007/06...,0,136,0,1,180,IN,1234889592,1306387514,865612499
1,1,587308,C,0,4,17183,0,Windows Vista to XP Downgrading/Reformat?,but my does not have where to off the SATA. wh...,0,136,0,1,180,unkwown,1304428703,1306387514,1949389026
2,2,587682,C,0,4,8981,0,Windows Vista to XP Downgrading/Reformat?,windows 7 (very similar to windows vista) won'...,0,136,0,1,180,unkwown,1304540935,1306387514,1017830945
3,3,614161,C,0,4,6930,0,Windows Vista to XP Downgrading/Reformat?,any body in this forum having problem of doung...,0,136,0,1,180,unkwown,1316594311,1306387514,3324437120
4,4,4,Q,0,4,0,1,Windows Vista to XP Downgrading/Reformat?,""",""Hello,\n\\n<br>I bought a new laptop with W...",0,136,1,1,180,FR,1199900147,1306387514,418598184


In [21]:
df_q_a.shape

(715516, 18)

In [22]:
df_q_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715516 entries, 0 to 715515
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Unnamed: 0        715516 non-null  int64 
 1   id                715516 non-null  int64 
 2   type              715516 non-null  object
 3   is_best_answer    715516 non-null  int64 
 4   topic_id          715516 non-null  int64 
 5   parent_id         715516 non-null  int64 
 6   votes             715516 non-null  int64 
 7   title             715509 non-null  object
 8   content           715378 non-null  object
 9   member            715516 non-null  int64 
 10  category          715516 non-null  int64 
 11  state             715516 non-null  int64 
 12  is_solved         715516 non-null  int64 
 13  num_answers       715516 non-null  int64 
 14  country           715512 non-null  object
 15  date              715516 non-null  int64 
 16  last_answer_date  715516 non-null  int

In [23]:
df_q_a.is_best_answer.value_counts()

0    686077
1     29439
Name: is_best_answer, dtype: int64

In [0]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [0]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [0]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [0]:

tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [0]:
df_q_a=df_q_a[df_q_a['content'].notnull()]

In [29]:
df_q_a.shape

(715378, 18)

In [0]:
from sklearn.model_selection import train_test_split

X = df_q_a['content'] # the features we want to analyze
ylabels = df_q_a['is_best_answer'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.3)

In [31]:
X_train.head

<bound method NDFrame.head of 18359     i can not remove it from the control panal... ...
171202    ","Dear Shams, \n\\n<br />\n\\n<br />While any...
188144    ","Hello,\n\\nbr />please can you kindly help ...
416500                                          it is goood
120908    ","HI.. I have a problem with CS 1.6 on Vista ...
                                ...                        
460469    ","Check All of your connections. Double check...
466879    ","Good morning all, \n\\n<br />\n\\n<br />\n\...
428461    ","Hello there. I get these message everytime ...
643463    I forget my Facebook old number I remember a p...
584774    ","Hello, \n\\n<br />\n\\n<br />\n\\n<br />\n\...
Name: content, Length: 500764, dtype: object>

In [0]:
# Logistic Regression Classifier
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error


from sklearn.ensemble import RandomForestClassifier
#classifier = LogisticRegression()
#classifier=xgb.XGBRegressor(n_estimators=1000)
classifier=RandomForestClassifier(n_estimators = 1000, random_state = 42)

# Create pipeline using Bag of Words
pipe = Pipeline([("cleaner", predictors()),
                 ('vectorizer', bow_vector),
                 ('classifier', classifier)])

# model generation
pipe.fit(X_train,y_train)

In [0]:
from sklearn import metrics
# Predicting with a test dataset
predicted = pipe.predict(X_test)

# Model Accuracy
print("RF Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("RF Regression Precision:",metrics.precision_score(y_test, predicted))
print("RF Regression Recall:",metrics.recall_score(y_test, predicted))

In [41]:
from sklearn.externals import joblib
joblib.dump(pipe, 'pipeline.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


['pipeline.pkl']

In [0]:
  drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
